# 15.095 | Homework 3

## Rachit Jain | 959080873

# Question 1

In [23]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames, Statistics

In [2]:
X_train = CSV.read("./UltimateFinal/Regression/X_train.csv", DataFrame);
X_train = X_train[:, 4:end]

Row,Arts & Photography,Biographies & Memoirs,Business & Money,Childrens Books,Christian Books & Bibles,Comics & Graphic Novels,Computers & Technology,Cookbooks,Crafts,Education & Teaching,Engineering & Transportation,Health,History,Humor & Entertainment,Law,Lesbian,Literature & Fiction,Medical Books,Mystery,Parenting & Relationships,Politics & Social Sciences,Reference,Religion & Spirituality,Romance,Science & Math,Science Fiction & Fantasy,SelfHelp,Sports & Outdoors,Teen & Young Adult,Travel,Avg Price,Rank,Price,Category_Arts & Photography,Category_Biographies & Memoirs,Category_Business & Money,Category_Childrens Books,Category_Christian Books & Bibles,Category_Comics & Graphic Novels,Category_Computers & Technology,Category_Cookbooks,Category_Crafts,Category_Education & Teaching,Category_Engineering & Transportation,Category_Health,Category_History,Category_Humor & Entertainment,Category_Law,Category_Lesbian,Category_Literature & Fiction,Category_Medical Books,Category_Mystery,Category_Parenting & Relationships,Category_Politics & Social Sciences,Category_Reference,Category_Religion & Spirituality,Category_Romance,Category_Science & Math,Category_Science Fiction & Fantasy,Category_SelfHelp,Category_Sports & Outdoors,Category_Teen & Young Adult,Category_Travel
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,10.6158,846491,13.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.985,4476208,12.95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.8,3.46154,0.0,0.0,0.0,0.0,0.0,0.0,3.66667,0.0,0.0,0.0,0.0,5.0,0.0,13.2515,932062,12.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.85714,0.0,4.0,0.0,0.0,0.0,0.0,4.2,0.0,4.625,0.0,0.0,4.75,0.0,12.2389,578404,14.95,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.53846,0.0,0.0,0.0,0.0,0.0,0.0,3.8,0.0,4.5,0.0,0.0,3.93548,0.0,12.4622,23939,9.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.55556,0.0,5.0,0.0,0.0,0.0,0.0,4.41667,0.0,4.5,0.0,0.0,0.0,0.0,13.1257,277763,13.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.90323,0.0,5.0,0.0,0.0,0.0,0.0,4.75,0.0,5.0,0.0,0.0,0.0,0.0,13.2485,2927209,9.95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,10.132,57728,12.78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,3.33333,4.03846,3.55556,0.0,0.0,0.0,0.0,4.08333,0.0,0.0,3.0,3.85714,4.5,4.0,0.0,0.0,3.625,4.25,0.0,5.0,4.16667,3.33333,3.0,0.0,3.9,0.0,3.0,0.0,0.0,4.0,17.0551,2641132,8.97,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
X_test = CSV.read("./UltimateFinal/Regression/X_test.csv", DataFrame);
X_test = X_test[:, 4:end]

Row,Arts & Photography,Biographies & Memoirs,Business & Money,Childrens Books,Christian Books & Bibles,Comics & Graphic Novels,Computers & Technology,Cookbooks,Crafts,Education & Teaching,Engineering & Transportation,Health,History,Humor & Entertainment,Law,Lesbian,Literature & Fiction,Medical Books,Mystery,Parenting & Relationships,Politics & Social Sciences,Reference,Religion & Spirituality,Romance,Science & Math,Science Fiction & Fantasy,SelfHelp,Sports & Outdoors,Teen & Young Adult,Travel,Avg Price,Rank,Price,Category_Arts & Photography,Category_Biographies & Memoirs,Category_Business & Money,Category_Childrens Books,Category_Christian Books & Bibles,Category_Comics & Graphic Novels,Category_Computers & Technology,Category_Cookbooks,Category_Crafts,Category_Education & Teaching,Category_Engineering & Transportation,Category_Health,Category_History,Category_Humor & Entertainment,Category_Law,Category_Lesbian,Category_Literature & Fiction,Category_Medical Books,Category_Mystery,Category_Parenting & Relationships,Category_Politics & Social Sciences,Category_Reference,Category_Religion & Spirituality,Category_Romance,Category_Science & Math,Category_Science Fiction & Fantasy,Category_SelfHelp,Category_Sports & Outdoors,Category_Teen & Young Adult,Category_Travel
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,3.68,0.0,4.28571,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,3.0,0.0,0.0,12.5656,3161695,17.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.33333,4.4,0.0,3.92857,4.0,3.0,0.0,4.0,0.0,0.0,0.0,5.0,4.125,4.5,0.0,0.0,3.61765,0.0,3.2,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.5,14.0012,3161695,17.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.91667,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,14.1564,3161695,17.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.625,0.0,0.0,0.0,4.375,0.0,0.0,0.0,3.18182,3.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,16.6236,3161695,17.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,4.5,4.16667,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.84,0.0,4.19355,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,4.0,17.7077,3161695,17.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,4.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.875,0.0,0.0,0.0,3.66667,0.0,3.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,21.6636,3161695,17.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0.0,4.21429,3.0,5.0,3.0,4.6,0.0,5.0,0.0,0.0,0.0,4.33333,0.0,4.5,0.0,0.0,4.0,0.0,4.2,0.0,1.0,4.5,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,13.9461,3502856,6.65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,4.0,3.85714,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.04,3.66667,0.0,0.0,4.0,0.0,3.5,5.0,3.72222,5.0,0.0,0.0,4.33333,3.5,0.0,3.25,0.0,0.0,14.4181,3502856,6.65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,4.0,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.55556,3.5,0.0,0.0,3.31818,0.0,3.31579,0.0,4.5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,21.7801,2021974,3.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
y_train = CSV.read("./UltimateFinal/Regression/y_train.csv", DataFrame);
y_train = y_train[:, 2]

562338-element Vector{Int64}:
 5
 5
 4
 5
 5
 5
 5
 5
 3
 5
 2
 2
 5
 ⋮
 5
 4
 5
 3
 4
 4
 4
 4
 5
 5
 1
 5

In [11]:
y_test = CSV.read("./UltimateFinal/Regression/y_test.csv", DataFrame);
y_test = y_test[:, 2]

48899-element Vector{Int64}:
 2
 4
 4
 5
 5
 4
 4
 4
 5
 5
 5
 5
 4
 ⋮
 4
 5
 5
 5
 4
 2
 5
 4
 2
 5
 4
 5

## Part A

The Lasso problem is:

$$\min_{\beta} \quad \| y-X\beta\|_2^2 + \lambda \|\beta\|_1.$$

We rewrite the above problem as:
$$\min_{\beta,t,a} \quad t + \rho \sum_{j=1}^p a_j
\quad \text{s.t.} \\
\quad \| y-X\beta\|_2^2 \leq t,\\
\quad \beta_j \leq a_j,\\
\quad -\beta_j \leq a_j.$$

We begin by implementing an optimization formulation for the $\ell_1$-regularized regression (Lasso):

In [13]:
function l1_regression(X, y, lambda; solver_output=0) # any argument after the semi-colon are the ones with default values
    n,p = size(X)
    
    # Build model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    # Insert variables
    @variable(model, beta[j=1:p])
    @variable(model, a[j=1:p]>=0)
    @variable(model, t>=0)
    
    #Insert constraints
    @constraint(model,[j=1:p], beta[j]<=a[j])
    @constraint(model,[j=1:p], -beta[j]<=a[j])
    @constraint(model, sum((y[i]-sum(X[i,j]*beta[j] for j=1:p))^2 for i=1:n) <= t)
    
    #Objective
    @objective(model, Min, t + lambda*sum(a[j] for j=1:p))
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
end

l1_regression (generic function with 1 method)

In [14]:
function compute_mse(X, y, beta)
    n,p = size(X)
    return sum((X*beta .- y).^2)/n
end

compute_mse (generic function with 1 method)

In [20]:
function compute_mae(X, y, beta)
    n,p = size(X)
    return mean(abs.(X*beta .- y))
end

compute_mae (generic function with 1 method)

In [28]:
function compute_r2(X, y_train, y, beta)
    num = sum((X*beta .- y).^2)
    den = sum((y .- mean(y_train)).^2)
    return 1-(num/den)
end

compute_r2 (generic function with 2 methods)

In [15]:
X_train = Matrix(X_train)
X_test = Matrix(X_test)

48899×63 Matrix{Float64}:
 0.0      3.0      0.0      3.0      …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 3.33333  4.4      0.0      3.92857     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0      0.0      0.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0      4.0      4.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.5      4.16667  4.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.0      3.6      0.0      0.0      …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0      4.21429  3.0      5.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.0      3.85714  4.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.0      4.4      0.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0      5.0      0.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0      4.25     0.0      0.0      …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0      3.0      0.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0      3.57143  0.0      0.0         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮          

In [16]:
beta_ols = l1_regression(X_train, y_train, 0.25);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


### In-Sample Metrics

In [30]:
is_mse = compute_mse(X_train, y_train, beta_ols)
print("The testing MSE is: ", is_mse)

The testing MSE is: 0.8261465169766279

In [31]:
is_mae = compute_mae(X_train, y_train, beta_ols)
print("The testing MAE is: ", is_mae)

The testing MAE is: 0.7094839546349411

In [32]:
is_r2 = compute_r2(X_train, y_train, y_train, beta_ols)
print("The testing R^2 is: ", is_r2)

The testing R^2 is: 0.09288076300320558

### Out-Of-Sample Metrics

In [21]:
oos_mse = compute_mse(X_test, y_test, beta_ols)
print("The testing MSE is: ", oos_mse)

The testing MSE is: 0.8342191674984312

In [24]:
oos_mae = compute_mae(X_test, y_test, beta_ols)
print("The testing MAE is: ", oos_mae)

The testing MAE is: 0.7116342820328428

In [29]:
oos_r2 = compute_r2(X_test, y_train, y_test, beta_ols)
print("The testing R^2 is: ", oos_r2)

The testing R^2 is: 0.09017482222933826

## Holistic Regression

In [35]:
corr_matrix = cor(X_train);

In [36]:
## Let us find the features that have a correlation of less than 0.5 or greater than -0.5. 
## This limits us to features that have low correlation.
corr_matrix_5 = (corr_matrix .<= 0.5) .& (corr_matrix .>= -0.5)
# corr_matrix_5 = LowerTriangular(corr_matrix_5) - Diagonal(corr_matrix_5)

63×63 BitMatrix:
 0  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  0  1  1  1  1  1  1  1  1  1  1  0     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  0  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  0  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  0  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  0  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  0  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  0  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  0  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  0  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  0  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  0  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  0  1  1  1  1  1  1  1  1  1  1  0     1  1

### Creating additional features

In [38]:
function create_features(X)
    
    ## Define constants
    n = size(X)[1]
    p = size(X)[2]
    
    ## Create an empty matrix with the necessary size to accomodate new features.
    X_new = zeros(n, 3*p);
    
    ## Creating additional features.
    for j in 1:p
        X_new[:, 3*(j-1) + 1] = X[:, j]
        X_new[:, 3*(j-1) + 2] = X[:, j].^2
        X_new[:, 3*(j-1) + 3] = abs.(X[:, j]).^0.5
    end
    
    return X_new
end

create_features (generic function with 1 method)

In [39]:
## Creating the modified X matrices.
X_train_new = create_features(X_train);
X_test_new = create_features(X_test);

In [40]:
## Sanity Check.
size(X_train_new)

(562338, 189)

### Create HC (Set of Highly Correlated Features)

In [41]:
corr_matrix = cor(X_train_new)
corr_matrix = (corr_matrix .>= 0.5) .| (corr_matrix .<= -0.5)
corr_matrix = LowerTriangular(corr_matrix) - Diagonal(corr_matrix)

189×189 LowerTriangular{Int64, Matrix{Int64}}:
 0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  1  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  1  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  1  1  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  0  0  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  0  0  1  0  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  0  0  1  1  0  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  0  0  0  0  0  0  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  0  0  0  0  0  1  0  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  0  0  0  0  0  1  1  0  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 0  0  0  0  0  0

In [42]:
## Number of pairs of highly correlated features
sum(corr_matrix)

216

### Create Holistic Regression Model

In [60]:
function holistic_regression(X, y, lambda; solver_output=0) # any argument after the semi-colon are the ones with default values

    ## Instantiating new variables.
    n, p_new = size(X)
    p = round(Int, p_new/3) # Making it an integer is essential.
    M = 50
    
    corr_matrix = cor(X)
    corr_matrix = (corr_matrix .>= 0.5) .| (corr_matrix .<= -0.5)
    corr_matrix = LowerTriangular(corr_matrix) - Diagonal(corr_matrix)
    
    corr_features_5 = sort(Tuple.(findall(>(0), corr_matrix)))
    HC_size = size(corr_features_5)[1]
    
    # Build model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    
    # Insert Variables.
    @variable(model, beta[j=1:p_new])
    @variable(model, a[j=1:p_new]>=0)
    @variable(model, t>=0)
    @variable(model, z[1:p_new], Bin)
    
    
    # Old Constraints.
    @constraint(model, [j=1:p], beta[j]<=a[j])
    @constraint(model, [j=1:p], -beta[j]<=a[j])
    @constraint(model, sum((y[i]-sum(X[i,j]*beta[j] for j=1:p_new))^2 for i=1:n) <= t)
    
    # New Constraints.
    @constraint(model, [j=1:p_new], -M*z[j] <= beta[j])
    @constraint(model, [j=1:p_new], beta[j] <= M*z[j])
    @constraint(model, sum(z[j] for j=1:p_new) <= 30)
    @constraint(model, [j=1:p], sum(z[i] for i=(3*(j-1)+1):(3*(j-1)+3)) <= 1)
    @constraint(model, [j=1:HC_size], sum(z[corr_features_5[j][i]] for i=1:2) <= 1)

    
    #Objective
    @objective(model, Min, t + lambda*sum(a[j] for j=1:p))
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))

end

holistic_regression (generic function with 1 method)

In [61]:
beta_holistic = holistic_regression(X_train, y_train, 0.25);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


We find the number of non-zero values in the resultant beta factor to be 8. This validates that the model ran correctly. Also, all the beta values are not zero which is another validation.

In [62]:
unique(beta_holistic)

16-element Vector{Float64}:
  0.0
  0.3251787299785467
 -0.09711819174844627
  0.06283338450881724
  0.4530664781528831
 -4.2747174390466335e-7
  0.8417127463335456
  5.32671406173109
  3.0961652540593545
 -4.4730249175299415
  6.116909237869239
  1.3665486306152383
  2.132689441522278
  3.26889511158588
  0.03068350269073564
 11.337106860459684

### In-Sample Metrics

In [63]:
is_mse = compute_mse(X_train, y_train, beta_holistic)
print("The testing MSE is: ", is_mse)

The testing MSE is: 5.169877084890784

In [64]:
is_mae = compute_mae(X_train, y_train, beta_holistic)
print("The testing MAE is: ", is_mae)

The testing MAE is: 1.780248277158238

In [65]:
is_r2 = compute_r2(X_train, y_train, y_train, beta_holistic)
print("The testing R^2 is: ", is_r2)

The testing R^2 is: -4.676589878724882

### Out-Of-Sample Metrics

In [66]:
oos_mse = compute_mse(X_test, y_test, beta_holistic)
print("The testing MSE is: ", oos_mse)

The testing MSE is: 5.173424543525212

In [67]:
oos_mae = compute_mae(X_test, y_test, beta_holistic)
print("The testing MAE is: ", oos_mae)

The testing MAE is: 1.7695749510477459

In [68]:
oos_r2 = compute_r2(X_test, y_train, y_test, beta_holistic)
print("The testing R^2 is: ", oos_r2)

The testing R^2 is: -4.6422965191635726

### Recommendations

In [118]:
books = CSV.read("./UltimateFinal/Books.csv", DataFrame);

In [119]:
X_train_copy = CSV.read("./UltimateFinal/Regression/X_train.csv", DataFrame);
X_test_copy = CSV.read("./UltimateFinal/Regression/X_test.csv", DataFrame);

In [122]:
y_train_pred_ols = X_train*beta_ols
y_train_pred_holistic = X_train*beta_holistic
X_train_copy[:,"regression_predictions"] = y_train_pred_ols
X_train_copy[:,"holistic_predictions"] = y_train_pred_holistic

y_test_pred_ols = X_test*beta_ols
y_test_pred_holistic = X_test*beta_holistic
X_test_copy[:,"regression_predictions"] = y_test_pred_ols
X_test_copy[:,"holistic_predictions"] = y_test_pred_holistic

48899-element Vector{Float64}:
  1.2116710815916356
  2.162121898834308
  3.100895960298091
  1.1463157236813868
  3.0759655909997354
  1.2073353651919216
  2.545125637306036
  0.6843006288202803
  1.5812140281796192
  0.9759766242812724
  2.8644354468399302
  1.153346840633207
  2.510362531274774
  ⋮
  2.666205709365917
  0.9489920409019597
  2.41287506053379
  2.297937009073404
  3.5200630838368845
  2.2065171059930195
  3.799793503721421
  3.5996345375261205
  0.7961048233113072
  1.31133275322081
 -2.855112858161844
  1.6698463971767321

In [123]:
X_copy = [X_train_copy; X_test_copy]

Row,Column1,User,Item,Arts & Photography,Biographies & Memoirs,Business & Money,Childrens Books,Christian Books & Bibles,Comics & Graphic Novels,Computers & Technology,Cookbooks,Crafts,Education & Teaching,Engineering & Transportation,Health,History,Humor & Entertainment,Law,Lesbian,Literature & Fiction,Medical Books,Mystery,Parenting & Relationships,Politics & Social Sciences,Reference,Religion & Spirituality,Romance,Science & Math,Science Fiction & Fantasy,SelfHelp,Sports & Outdoors,Teen & Young Adult,Travel,Avg Price,Rank,Price,Category_Arts & Photography,Category_Biographies & Memoirs,Category_Business & Money,Category_Childrens Books,Category_Christian Books & Bibles,Category_Comics & Graphic Novels,Category_Computers & Technology,Category_Cookbooks,Category_Crafts,Category_Education & Teaching,Category_Engineering & Transportation,Category_Health,Category_History,Category_Humor & Entertainment,Category_Law,Category_Lesbian,Category_Literature & Fiction,Category_Medical Books,Category_Mystery,Category_Parenting & Relationships,Category_Politics & Social Sciences,Category_Reference,Category_Religion & Spirituality,Category_Romance,Category_Science & Math,Category_Science Fiction & Fantasy,Category_SelfHelp,Category_Sports & Outdoors,Category_Teen & Young Adult,Category_Travel,regression_predictions,holistic_predictions
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64
1,0,1,22721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,10.6158,846491,13.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.52238,4.53294
2,1,2,15742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.985,4476208,12.95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.21803,1.07899
3,2,3,20373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.8,3.46154,0.0,0.0,0.0,0.0,0.0,0.0,3.66667,0.0,0.0,0.0,0.0,5.0,0.0,13.2515,932062,12.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.99713,4.35907
4,3,4,15233,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.85714,0.0,4.0,0.0,0.0,0.0,0.0,4.2,0.0,4.625,0.0,0.0,4.75,0.0,12.2389,578404,14.95,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.6844,3.48425
5,4,5,1644,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.53846,0.0,0.0,0.0,0.0,0.0,0.0,3.8,0.0,4.5,0.0,0.0,3.93548,0.0,12.4622,23939,9.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4.00966,2.92344
6,5,6,20734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.55556,0.0,5.0,0.0,0.0,0.0,0.0,4.41667,0.0,4.5,0.0,0.0,0.0,0.0,13.1257,277763,13.99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.39837,4.63153
7,6,7,20608,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.90323,0.0,5.0,0.0,0.0,0.0,0.0,4.75,0.0,5.0,0.0,0.0,0.0,0.0,13.2485,2927209,9.95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4.48081,2.47582
8,7,8,10625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,10.132,57728,12.78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5229,2.96777
9,8,9,21522,3.33333,4.03846,3.55556,0.0,0.0,0.0,0.0,4.08333,0.0,0.0,3.0,3.85714,4.5,4.0,0.0,0.0,3.625,4.25,0.0,5.0,4.16667,3.33333,3.0,0.0,3.9,0.0,3.0,0.0,0.0,4.0,17.0551,2641132,8.97,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.10607,-0.0854636


### For Regression

In [111]:
user_9512 = filter(:User => n -> n == 9512, X_copy)
top_pred_books_9512 = sort(user_9512, :regression_predictions, rev=true)[1:10,"Item"]
book_recommendations_9512 = filter(row -> row.Item in top_pred_books_9512, books)
# book_recommendations_9512.order=[findfirst(book_recommendations_9512.Item .== x) for x in top_pred_books_9512]
# sort(book_recommendations_9512, :order)
CSV.write("./UltimateFinal/Recommendations/9512/l1_regression.csv", book_recommendations_9512)

"./UltimateFinal/Recommendations/9512/l1_regression.csv"

In [115]:
user_53 = filter(:User => n -> n == 53, X_copy)
top_pred_books_53 = sort(user_53, :regression_predictions, rev=true)[1:10,"Item"]
book_recommendations_53 = filter(row -> row.Item in top_pred_books_53, books)
# book_recommendations_9512.order=[findfirst(book_recommendations_9512.Item .== x) for x in top_pred_books_9512]
# sort(book_recommendations_9512, :order)
CSV.write("./UltimateFinal/Recommendations/53/l1_regression.csv", book_recommendations_53)

"./UltimateFinal/Recommendations/53/l1_regression.csv"

In [124]:
# book_recommendations_53

### For Holistic Regression

In [125]:
user_9512 = filter(:User => n -> n == 9512, X_copy)
top_pred_books_9512 = sort(user_9512, :holistic_predictions, rev=true)[1:10,"Item"]
book_recommendations_9512 = filter(row -> row.Item in top_pred_books_9512, books)
# book_recommendations_9512.order=[findfirst(book_recommendations_9512.Item .== x) for x in top_pred_books_9512]
# sort(book_recommendations_9512, :order)
CSV.write("./UltimateFinal/Recommendations/9512/holistic_regression.csv", book_recommendations_9512)

"./UltimateFinal/Recommendations/9512/holistic_regression.csv"

In [126]:
user_53 = filter(:User => n -> n == 53, X_copy)
top_pred_books_53 = sort(user_53, :holistic_predictions, rev=true)[1:10,"Item"]
book_recommendations_53 = filter(row -> row.Item in top_pred_books_53, books)
# book_recommendations_9512.order=[findfirst(book_recommendations_9512.Item .== x) for x in top_pred_books_9512]
# sort(book_recommendations_9512, :order)
CSV.write("./UltimateFinal/Recommendations/53/holistic_regression.csv", book_recommendations_53)

"./UltimateFinal/Recommendations/53/holistic_regression.csv"

In [128]:
# book_recommendations_53